In [1]:
import tensorflow as tf
from tensorflow.keras.models import model_from_json

import os
import numpy as np
import mtcnn
import cv2
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
# # Load the model architecture from the JSON file
# with open('face_net_model.json', 'r') as json_file:
#     loaded_model_json = json_file.read()

# face_net_model = model_from_json(loaded_model_json)

# # Load the model weights
# face_net_model.load_weights("face_net_model_weights.h5")

In [2]:
face_net_model = tf.keras.models.load_model('model.tf')

In [4]:
face_net_model.summary()

Model: "FaceNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 SiameseModel (Functional)   (None, None)              26282496  
                                                                 
Total params: 26282498 (100.26 MB)
Trainable params: 2691712 (10.27 MB)
Non-trainable params: 23590786 (89.99 MB)
_________________________________________________________________


In [5]:
face_net_model.siamese_model.get_layer(name='embedding').summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 1, 1, 2048)        0         
 ng2D)                                                           
                                                                 
 dense_8 (Dense)             (None, 1, 1, 256)         524544    
                                                                 
 batch_normalization_192 (B  (None, 1, 1, 256)         1024      
 atchNormalization)                                              
                                                                 
 dense_9 (Dense)             (None, 1, 1, 128)         32

In [5]:
detector = mtcnn.MTCNN()
cap = cv2.VideoCapture(1)

while cap.isOpened() :
    ret, frame = cap.read()

    faces = detector.detect_faces(frame)

    # Draw bounding boxes around detected faces
    for result in faces:
        x, y, width, height = result['box']
        cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
    
    cv2.imshow('Image', frame)
    
    
    if cv2.waitKey(1) & 0xFF == ord('q') :
        break
cap.release()
cv2.destroyAllWindows()

In [3]:
embedding_size = face_net_model.siamese_model.get_layer('embedding').output_shape[-1]
database = pd.DataFrame([], index = pd.Index([], name='Name'), columns = range(embedding_size), dtype=np.float64)
database

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
Name,,,,,,,,,,,,,,,,,,,,,


In [11]:
detector = cv2.CascadeClassifier(os.path.join('HaarCascade', 'frontalface_default.xml'))
cap = cv2.VideoCapture(0)
PADDING = 0

data_dir = os.path.join('data', 'self data')
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
while cap.isOpened() :
    ret, frame = cap.read()
    frame_copy = frame.copy()
       
    grayscaled_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    #detect faces
    face_coordinates = detector.detectMultiScale(grayscaled_img)

    #draw rectangles around the faces
    for (x, y, w, h) in face_coordinates:
        cv2.rectangle(frame, (x, y), (x + w, y + h), 
                      (128, 128, 128), 
                      2)

    cv2.imshow('Image', frame)

        
    if cv2.waitKey(1) & 0xFF == ord(' '):
        # Take input for the name
        input_name = input("Enter name for this frame: ")
        # Save the frame with the given name
        resized_face = cv2.resize(frame_copy[y-PADDING//2:y+h+PADDING//2, x-PADDING//2:x+w+PADDING//2], (160, 160))
        cv2.imwrite(os.path.join(data_dir, input_name + '.jpg'), resized_face)
        embedding_layer_model = face_net_model.siamese_model.get_layer(name='embedding')
        pre_processed_img = np.expand_dims(resized_face/255.0, 0)
        embedding_layer_output = embedding_layer_model.predict(pre_processed_img)
        database.loc[input_name] = tf.squeeze(embedding_layer_output).numpy()
        print(f"Frame saved as {input_name}.jpg")
        
    if cv2.waitKey(1) & 0xFF == ord('q') :
        break
        
cap.release()
cv2.destroyAllWindows()

Enter name for this frame:  scarlett


1/1 [==============================] - 0s 142ms/step
Frame saved as scarlett.jpg


In [12]:
database

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
Name,,,,,,,,,,,,,,,,,,,,,
dhruv,3.113243,-0.307245,0.177253,2.534306,1.002873,1.496851,1.052221,0.907640,-2.816007,-1.876787,...,-0.303094,0.127854,-1.292186,-0.152681,1.204967,-3.657726,2.758280,2.259524,-3.529215,-2.873884
rdj,0.049351,-1.388118,1.204812,1.207747,0.113464,0.010652,-0.960037,1.802148,-2.422195,-1.915690,...,-1.611210,-1.359364,0.656045,-0.452269,1.085845,-0.343031,3.006766,-0.749430,-1.086820,-0.838277
scarlett,-0.136039,0.456802,1.648346,0.361069,0.537668,-0.425806,-0.767333,2.689404,-2.271242,-1.598032,...,-1.103851,-3.096128,-0.098867,-0.507460,0.497919,-0.642437,2.615344,-0.858801,-0.805155,0.076372


In [13]:
if not os.path.exists(os.path.join('data', 'database')) :
    os.makedirs(os.path.join('data', 'database'))
database.to_csv(os.path.join('data', 'database', 'database-1.csv'))

In [18]:
database = pd.read_csv(os.path.join('data', 'database', 'database-1.csv'))
database = database.set_index('Name')
database

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
Name,,,,,,,,,,,,,,,,,,,,,
dhruv,3.113243,-0.307245,0.177253,2.534306,1.002873,1.496851,1.052221,0.907640,-2.816007,-1.876787,...,-0.303094,0.127854,-1.292186,-0.152681,1.204967,-3.657726,2.758280,2.259524,-3.529215,-2.873884
rdj,0.049351,-1.388118,1.204812,1.207747,0.113464,0.010652,-0.960037,1.802148,-2.422195,-1.915690,...,-1.611210,-1.359364,0.656045,-0.452269,1.085845,-0.343031,3.006766,-0.749430,-1.086820,-0.838277
scarlett,-0.136039,0.456802,1.648346,0.361069,0.537668,-0.425806,-0.767333,2.689404,-2.271242,-1.598032,...,-1.103851,-3.096128,-0.098867,-0.507460,0.497919,-0.642437,2.615344,-0.858801,-0.805155,0.076372


In [21]:
PADDING = 0
def l2_similarity(embedding1, embedding2):
    return np.sqrt(np.sum(np.square(embedding1 - embedding2)))

# Load Haar Cascade classifier for face detection
detector = cv2.CascadeClassifier(os.path.join('HaarCascade', 'frontalface_default.xml'))

# Open the camera
cap = cv2.VideoCapture(1)

# Define threshold for similarity
threshold = face_net_model.threshold.numpy()  # Adjust as needed

def is_face_in_database(embedding):
    min_similarity = float('inf')  # Initialize minimum similarity to positive infinity
    matching_label = None  # Initialize matching label to None
    
    for index, row in database.iterrows():
        db_embedding = row.values
        similarity = l2_similarity(embedding, db_embedding)
        
        if similarity < min_similarity:
            min_similarity = similarity
            matching_label = index
        
        # if similarity < threshold :
        #     matching_label = index
        #     break
    
    return matching_label

while cap.isOpened():
    ret, frame = cap.read()
    
    # Convert the frame to grayscale
    grayscaled_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    face_coordinates = detector.detectMultiScale(grayscaled_img)

    # Draw rectangles around the faces and label them
    for (x, y, w, h) in face_coordinates:
        # 
        roi_x1 = max(x - PADDING//2, 0)
        roi_y1 = max(y - PADDING//2, 0)
        roi_x2 = min(x + w + PADDING//2, frame.shape[1])
        roi_y2 = min(y + h + PADDING//2, frame.shape[0])

        # Crop and resize face region
        resized_face = cv2.resize(frame[roi_y1:roi_y2, roi_x1:roi_x2], (160, 160))

                
        # Obtain embedding for the face using the model
        embedding_layer_model = face_net_model.siamese_model.get_layer(name='embedding')
        pre_processed_img = np.expand_dims(resized_face/255.0, 0)
        embedding = np.squeeze(embedding_layer_model.predict(pre_processed_img, verbose=0))
        
        # Check if face is in database
        name = is_face_in_database(embedding)
        
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (128, 128, 128), 2)
        
        # Plot name if face is recognized, otherwise plot "?"
        if name:
            cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        else:
            cv2.putText(frame, "?", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    # Display the frame
    cv2.imshow('Image', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

In [33]:
cap.release()
cv2.destroyAllWindows()